<a href="https://colab.research.google.com/github/AdmiralHonda/ml_intro/blob/main/ch02/treat_text_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
# テキストの扱いについて

ここではPythonでの文字を取り扱う補足を記しておきます。  
基本的にいつも自分が研究や個人製作で使っている方法をまとめたヒントだと思ってください。  
できる人の場合はコードレビューをしてもらい、こうしたほうが良いよ的なコメントをください。  
その場合はurlからgithub上で公開されていることがわかると思うので、isuueで指定してください。

### 目次
- 文字列を取り扱う
- 正規表現
- 変な文字を削除する
  > - 削除する対象
  > - python組み込みの削除するメソッド

In [ ]:
#@title 文字列を取り扱う(配列)

"""
機械学習でも解析でもなんでも基本的には以下のように文字列の配列
で表されることが多いです。
"""
sample_text_list = ["錬成により砲撃Lv8にできるようになった「カムラノ傑銃鎗」は、トップクラスの性能に躍り出ました。\n",\
                    "「闇討ち」が装飾品でつけられるようにもなり\n、スキルの自由度もアップ。\n",\
                    "錬成で「カムラノ傑銃鎗」を砲撃Lv8にし、扱いやすいガード型で運用する構成。\n",\
                    "堅実に立ち回りたい人におすすめです。\n"]

# 下処理などでよく使う記法
"""
ここでは前述した配列で解析対象のデータに対して何らかの処理を施す場合のヒントを記します。
先ほどのsample_text_listに対していろいろ試してみましょう
簡単な処理（先頭n文字を抽出など）の場合は以下のようにすると早く実行できます。
内包記法といっていちいちfor文で要素を取り出して新しい配列に格納するよりも高速です。
Pythonではなるべく素のfor文を使わないように心がけることが重要です。
"""
# new_sample_textにsample_text_listにある文の先頭５文字を抽出
new_sample_text = [ sentense[:5] for sentense in sample_text_list]
print(new_sample_text)

# 条件を指定して処理("錬成"という文字列が入ったオブジェクトのみ抽出)
# 内包記法の条件文は[]内のforで取り出した要素一つに対して何らかの判定を掛けます。
new_sample_text_if = [sentense[:5] for sentense in sample_text_list if "錬成" in sentense]
print(new_sample_text_if)


In [ ]:
#@title 文字列を取り扱う(辞書型)
"""
Pythonには辞書型のデータ型があります。ほかの言語では連想配列と呼ばれることがあります。
例えば
`名前 - ["文字","文字",...,"文字"]
    　- [数値,数値,..,数値]
`
のように名前に関連して二つのデータが合う場合は名前に対応した形で管理したい時に有効です。
自分の場合は研究でレシピのIDに対応して調理手順や食材などの情報を辞書型で管理しています。
機械学習でも、各単語をキーにしてベクトルを管理することがあります。
リストにして検索するよりも単一のデータへのアクセスが早いので便利です。

"""

# 辞書型の宣言と格納
equipments = {"リバルクシャナアンク" : ["風纏Lv1","翔蟲使いLv3"],\
                "リバルクシャイラム" : ["鋼殻の恩恵Lv2","攻撃Lv2","業物Lv2","弾丸節約Lv2"],\
                "リバルクシャナグラン" : ["風纏Lv1","連撃Lv2"],\
                "リバルクシャナアンダ" : ["鋼殻の恩恵Lv2","攻撃Lv2","体力回復量UPLv3"],\
                "リバルクシャナペイル" : ["風纏Lv1","連撃Lv1"]}

print(equipments["リバルクシャナアンダ"])

"""
辞書型のデータの処理

辞書型で管理し、それを用いる際に要素それぞれに何か処理を行いたい場合は以下のようにしてます。
この処理は結構多いです。機械学習の学習データセットは自分の場合はjson形式で管理しているので、皆さんも自分の研究を引き継ぐ際には目にすると思います。
"""

# 辞書をリスト型に変換し、要素をひとつづつ処理をしてリストに格納
# 辞書は.items()メソッドを使用すればリストになり、その要素はタプル（リストのようなもの）になり要素[0]がキー、要素[1]が中身のデータとなる
# "攻撃Lv2"という文字の入ったキーを検索している
skills = [equipment[0] for equipment in equipments.items() if "攻撃Lv2" in equipment[1]]
print(skills)

## 正規表現
文字のパターンを記述したものを[正規表現](https://developer.mozilla.org/ja/docs/Web/JavaScript/Guide/Regular_Expressions)といいます。  
`コンパイラおよび演習`を受けた方はもしかしたらきいたことはあるかもしれないですね。  
対象の文字列に検索を掛ける時に使います。また後述しますが、いらない記号などを削除する際にも使用します。  
ここでは正規表現に関してはそういった便利なものがある程度にしか書きません。詳しく知りたい方は[ここ](https://developer.mozilla.org/ja/docs/Web/JavaScript/Guide/Regular_Expressions)で学んでください。  
Pythonでも様々な場面で正規表現を使って検索、置換、削除を行うことができます。

In [18]:
#@title Pythonでの正規表現の例
import re

# "【攻撃力】340 【属性】龍50 【会心】-35%/15%"内をパターン([0-9]+で検索して数字の部分を返す。
print(re.findall("[0-9]+","【攻撃力】340 【属性】龍50 【会心】-35%/15%"))

['340', '50', '35', '15']


## 邪魔な文字を削除する

文字列処理ではいくつかの文字は分析結果を悪くする危険があるため、処理の前に取り除く工程をはさむ必要があります。  

以下にその例を示します。
- 改行、タブなどの特殊文字
  > いくつかの言語処理ツールは改行文字などで処理の単位の区切りとしたりする場合があり、意図していない形でデータが入る危険性があります。  
  特殊文字の例）"\n","\t","\r"
  これらが含まれそうな文章（webから盗んできたやつとか）はあらかじめ消す工程をはさみましょう
- 記号
  >記号は対象の単語の前後につくことによってうまく認識できないことがあります。  
  例)"★僕のとっておき","簡単♪お手軽レシピ！"  
  このような例でもあるように記号は読み手側には影響を与えますが、計算機にとっては意味的には何ら変わらないゴミとなります。  
  そしてアジア圏では言語を解析する際に単語に分割する処理があるのですが、時々この記号と目的の単語がまとめられることがあり、そうなると意味は同じなのに違う単語として扱われるケースがあります。  
  それを避けるためにも消しておきましょう。  

In [19]:
#@title 邪魔な文字に消えていただく

print("★僕の\nとっておき")

# .replaceを用いる場合
print("replace:★僕のとっておき".replace("★","").replace("\n",""))

# re.subを使う場合
print(re.sub("★|\n","","re.sub: ★僕の\nとっておき"))

★僕の
とっておき
replace:僕のとっておき
re.sub: 僕のとっておき


In [ ]:
#@title 課題
"""
先ほど入力したnew_sample_textから「」でくくられた文字列を抽出するスクリプトを作成せよ

期待する文字列：
    カムラノ傑銃鎗
    闇討ち
"""

